# Comparing Imputers on the ExtraSensory data set

## Set up the Notebook

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
sys.path.insert(0, os.path.abspath('..'))
os.chdir(os.path.abspath('..'))

#Disable multi-threading in NumPy 
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import mkl
mkl.get_max_threads()
mkl.set_num_threads(1)

## Import modules

In [ ]:
from Blocks.data_loader import extrasensory_data_loader
from Blocks.filter import MisingLabelFilter,  MisingDataColumnFilter, Take
from Blocks.imputer import Imputer
from Blocks.normalizer import Normalizer
from Blocks.experimental_protocol import ExpTrainTest, ExpCV, ExpWithin
from Blocks.results_analysis import ResultsConcat, ResultsCVSummarize, DataYieldReport

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.dummy import DummyClassifier

import matplotlib.pyplot as plt

from Workflow.workflow import workflow
import Workflow.compute_graph
import time
import pandas as pd


## Define the workflow

This workflow uses a basic train/test experiment to compare two different imputation methods: zero imputation and mean imputation. The model used is logistic regression with a fixed regularization hyper-parameter. 

The workflow includes a column filter that screens out feature dimensions that are less than 20% observed, and a missing label filter that removes instances without labels. Next, the workflow performs imputation in each of two differnt ways. For each branch of the workflow, normalization is applied, followed by the the train/test experiment. Data loading and filtering is shared across the two branches of the workflow. The workflow computation graph is then created and displayed along with the results. 

In this experiment, we can see that the use of mean imputation results in more than a 10% performance improvement relative to zero imputation.

In [ ]:
imp_methods = ["mean", "constant"]
metrics     = [accuracy_score, f1_score, precision_score, recall_score]

df_raw      = extrasensory_data_loader(label="SLEEPING");
df_cf       = MisingDataColumnFilter(df_raw);
df_lf       = MisingLabelFilter(df_cf);

res=[]
for i,m in enumerate(imp_methods):
    df_imp = Imputer(df_lf, method=m)
    df_norm = Normalizer(df_imp)
    models = {"LR(C=1e-4, %s)"%m: LogisticRegression(solver="lbfgs",max_iter=100)}
    res.append(ExpTrainTest(df_norm, models, metrics=metrics));

res_cat = ResultsConcat(*res)
flow    = workflow(res_cat)
 
flow.draw(); plt.show();
output=flow.run(backend="sequential", monitor=True, from_scratch=True);

In [ ]:
output["Results Concat"]["report"]